# Movie Analysis Project

* Project Contributors:  Raizel Bernstein and Steven Yan
* Project Supervisors:  Fangfang Lee and Justin Tannenbaum

# Web Scraping for Data Collection

## Import modules

We walk through step-by-step how we went about webscraping through the IMDB website with Beautiful Soup.  We import the following modules for webscraping: requests, time, random, and tqdm.  We import the following modules for data cleaning:  numpy and pandas.  Requests is the module we use for send the http requests to get the HTML files, which we use the module BeautifulSoup to parse.  Numpy is the module we use for certain mathematical operations and array manipulation.  Pandas is the module for organizing the data into a DataFrame, which facilitates data cleaning and analysis.  Time is a module for representing time in code and allows us to create pauses.during code execution, while random helps us to create pseudo-random numbers for the pauses in the code execution.  Tqdm creates the progress bar for executing code for API calling or webscraping.

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import random
from tqdm.notebook import trange, tqdm
from bs4 import BeautifulSoup as BS
import json

## Web Scraping through One Entry

We are going to walk through the process of working through a single page before the process of applying the same process to multiple pages.  We start with the URL that we want to scrape.  We make a GET request from the API of the website, and we receive a Response object in return.  We pass the response into the BeautifulSoup constructor, which parses the document with the HTML parser.  We are then able to view the HTML document as a complex tree of Python objects.  

In [2]:
url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=boxoffice_gross_us,desc'
response = requests.get(url)
soup = BS(response.text, "html.parser")



We identify the div class of lister-item mode-advanced to contain the information that we need.  We use the find() method to extract the first of the 50 div containers representing the top 50 of the 1000 on the webpage.  We try to put out the value of just one to make sure we have the right values.  We are grabbing movie titles, IMDB id's, PG ratings, the year of release, runtimes, genres, IMDB ratings, metascores, total number of votes submitted for IMDB rating, and gross US earnings.  

In [3]:
div = soup.find('div', {'class':'lister-item mode-advanced'})


In [4]:
title = div.h3.a.text
imdb_id = div.find('img')['data-tconst']
year = div.h3.find('span', class_='lister-item-year').text
pg_rating = div.find('span', class_='certificate').text
runtime = div.find('span', class_='runtime').text
genre = div.find('span', class_='genre').text
imdb_rating = div.strong.text
metascore = div.find('span', class_='metascore').text
nv = div.find_all('span', attrs={'name': 'nv'})
vote = nv[0].text
gross = nv[1].text


## Web Scraping through Multiple Entries on Multiple Pages

In order to iterate through all 50 instances and then to iterate through all 200 pages for the 1000 entries, we create a for loop to iterate through the 50 movies on each page and another for loop to iterate through each page.  First, we instantiate the variables that will represent each of the data we are gathering.

In [5]:
titles = []
imdb_ids = []
pg_ratings = []
years = []
runtimes = []
genres = []
imdb_ratings = []
metascores = []
votes = []
gross_us = []

To iterate through each page, we look at the URLs for each page containing 50 entries and find the pattern for iteration.  As we move past the 1st page through the pages, the value after start changes, from 51, 101, 151, 201, etc.  We postulate that if we substitute 1 we would get the original page.  We use string formatting to insert the different values we are iterating through.  We are iterating starting the range of 1 to 1000 at intervals of 50.  As we find the values, we append them to their respective lists.  When we started to run the for loops, we encountered errors.  We added to some of the original code to account for empty values to ensure all of our lists had 1000 values at the end.

In [6]:
for n in tqdm(range(1, 1000, 50), desc='Download Progress:'):
    # Create an expression that represents each page for the iteration
    url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=boxoffice_gross_us,desc&start={}&ref_=adv_nxt'.format(n)
    # The requests library makes a get request to the url for data, which is saved to results
    results = requests.get(url)
    # Create an instance of BeautifulSoup to parse results
    soup = BS(results.text, "html.parser")
    # Find the div container in the HTML that contains the wanted information.  
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    # Use function to suspend execution of calling thread at random
    time.sleep(random.randint(3, 12))

    # for each container in the div container created above by Beautiful Soup
    for div in movie_div:
        # to get the titles, we use attribute notation to access the title contained as text in the <a> tag nested inside the <h3> tag 
        # dot notation only works with the first instance of the tag
        title = div.h3.a.text
        # we append the scraped title to the titles list through each iteration
        titles.append(title)
        # to get the IMDB id's, we use the find() method to find the first instance 
        imdb_id = div.find('img')['data-tconst']
        imdb_ids.append(imdb_id)
        # to scrape the year, we use the find() method to search nested inside the h3 tag to get the text inside the span tag with the class lister-item-year
        year = div.h3.find('span', class_='lister-item-year').text
        years.append(year)
        # to scrape the pg rating, we use the find method again but create a condition in the case of blank values
        pg_rating = div.find('span', class_='certificate').text if div.p.find('span', class_='certificate') else '--'
        pg_ratings.append(pg_rating)
        # to scrape the runtime, we use a similar method from above
        runtime = div.find('span', class_='runtime').text if div.p.find('span', class_='runtime') else '--'
        runtimes.append(runtime)
        # to scrape the genres, we employed the same method
        genre = div.find('span', class_='genre').text
        genres.append(genre)
        # to scrape the IMDB rating, we call the distinctive strong tag which wraps the desired text
        imdb_rating = div.strong.text
        imdb_ratings.append(imdb_rating)
        # to scrape the etascore, we use a similar code as above
        metascore = div.find('span', class_='metascore').text if div.find('span', class_='metascore') else '--'
        metascores.append(metascore)
        # to scrape the votes and gross us earnings, we use the find_all method, which finds all the instances of the span tag with the name attribute and value of nv
        # if there is one item in the list, it represents the vote, and if there are two items, then we get the gross earning and return a string if it's empty to ensure all our lists are the same length to make the DataFrame
        nv = div.find_all('span', attrs={'name': 'nv'})
        vote = nv[0].text
        votes.append(vote)
        gross = nv[1].text if len(nv) > 1 else '--'
        gross_us.append(gross)



Download Progress::   0%|          | 0/20 [00:00<?, ?it/s]

In [9]:
import pickle
pickle.dump(results, open("data/imdb.pickle", "wb" ))


## Creating DataFrame for Data Cleaning

We then created a dataframe that would contain all the values of the lists with each column representing a different type of data.  This allows us to clean and manipulate the data more easily.

In [7]:
# Create a Pandas dataframe with the 
movies = pd.DataFrame({
    'movie': titles, 'year' : years, 'pg_rating' : pg_ratings, 'imdb_id' : imdb_ids, 'runtime' : runtimes, 'genre' : genres, 'metascore' : metascores, 'imdb_rating' : imdb_ratings, 'votes' : votes, 'gross_us' : gross_us
})


In [8]:
movies.to_csv("data/movies.csv")

## Cleaning the Web Scraped Data

To determine how the data needs to be cleaned, we called an entry from each columns to see the output. When the values are displayed inside the dataframe, we couldn't determine whether there was whitespace or not. We used df.dtypes to determine whch column values needed to be converted into the appropriate type.

In [10]:
# We used extract and regex to pull out any digits of length 1 or more and to turn that string into an integer type
movies['imdb_id'] = movies['imdb_id'].str.extract('(\d+)').astype(int)
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies['runtime'] = movies['runtime'].str.extract('(\d+)').astype(int)
# We used rstrip() to remove whitespace right of the metascore
movies['metascore'] = movies['metascore'].str.rstrip()
# Because of empty values, we can convert those non-numeric values to numeric with coerce
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')
# We used replace() to remove the comma from the vote count and turn the value into an integer
movies['votes'] = movies['votes'].str.replace(',', '').astype(int)
# We used map() and lambda function to apply the lstrip() method and rstrip() method
movies['gross_us'] = movies['gross_us'].map(lambda x: x.lstrip('$').rstrip('M'))
# We used map() and lambda function to strip \n in front and whitespace after
movies['genre'] = movies['genre'].map(lambda x: x.strip('\n').rstrip())

In [28]:
# Write dataframe to a CSV file
movies.to_csv('data/top_1000_by_us_box_office.csv', index=False)

# Making API requests for Data Collection

We collected another data set from TMDB using their sample URL requests tool to create the appropriate URL to find the top 1000 most popular movies.  The code is very similar to the webscraping one, except that we create a request using the Request() constructor to fetch a .json file.  We call the json() method to read and parse the data into a dictionary.  The for loop iterates through each of the pages and appends the data into compiled_list.


## Making API calls

In [12]:
compiled_list = []

for n in tqdm(list(range(1,399))):
    url='https://api.themoviedb.org/3/movie/top_rated?api_key=0e72c0b2b11293a6390e9f7b472aec2b&language=en-US&page={}'.format(n)
    r = requests.get(url)
    data = r.json()
    compiled_list.append(data)
    time.sleep(random.choice([1,2]))


  0%|          | 0/398 [00:00<?, ?it/s]

In [13]:
pickle.dump(compiled_list, open("data/tmdb.pickle", "wb" ))

In [16]:
movies_2 = []
for page in compiled_list:
    movies_2 += page['results']

## Creating DataFrame

In [17]:
df = pd.DataFrame(movies_2)

In [18]:
df.columns

Index(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'title', 'video', 'vote_average', 'vote_count'],
      dtype='object')

In [20]:
df = df.drop(columns = ['backdrop_path', 'poster_path', 'video', 'adult', 'original_title', 'overview'])

Upon inspecting the data, we realized we had to convert the genre ID's into the actual genre names.  We achieved that by creating dictionary of key:value pairs, where the key is the ID and value is the genre name.  We passed the dictionary through each of the rows of the genre column to replace the values.

In [21]:
df.head()

,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count
0,"[10749, 35]",761053,en,34.985,2020-11-19,Gabriel's Inferno Part III,8.9,703
1,[10749],724089,en,6.760,2020-07-31,Gabriel's Inferno Part II,8.8,1166
2,"[35, 18, 10749]",19404,hi,14.342,1995-10-20,Dilwale Dulhania Le Jayenge,8.8,2643
3,[10749],696374,en,11.163,2020-05-29,Gabriel's Inferno,8.7,1931
4,"[18, 80]",278,en,43.166,1994-09-23,The Shawshank Redemption,8.7,18322


In [22]:
genre = {28: 'Action', 12: "Adventure", 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}

In [23]:
for g_id in df['genre_ids']:
    for g in g_id:
        if g not in genre.keys():
            print(g)

In [24]:
df['genre_ids'] = df['genre_ids'].apply(lambda g_id: [genre[g] for g in g_id])

In [30]:
df.to_csv("data/tmdb_top_rated.csv", index=False)

## Combining data from two databases

We uploaded the CSV from the webscraping exercise, merged the two dataframes together, and inspected our resulting dataframe, and dropped extraneous columns.

In [31]:
top = pd.read_csv('data/top_1000_by_us_box_office.csv')
combined = pd.merge(df, top, left_on = 'title', right_on = 'movie')
combined.columns

Index(['genre_ids', 'id', 'original_language', 'popularity', 'release_date',
       'title', 'vote_average', 'vote_count', 'movie', 'year', 'pg_rating',
       'imdb_id', 'runtime', 'genre', 'metascore', 'imdb_rating', 'votes',
       'gross_us'],
      dtype='object')

In [32]:
combined = combined.drop(columns=['movie', 'metascore', 'vote_count', 'vote_average', 'original_language', 'id'])

We checked for any null values in the table, and dropped the rows which had null values for the gross earnings.  We also wanted to pull the month and the year out of the release date, so we could sort by month or year.  By checking dtypes, we saw that certain columns needed to be converted into integers.

In [33]:
combined.isna().sum()

genre_ids       0
popularity      0
release_date    0
title           0
year            0
pg_rating       0
imdb_id         0
runtime         0
genre           0
imdb_rating     0
votes           0
gross_us        0
dtype: int64

In [34]:
combined['month'] = combined['release_date'].apply(lambda m: m[5:7])

In [35]:
combined['year'] = combined['year'].apply(lambda x: int(x))

In [36]:
combined['month'] = combined['month'].apply(lambda x: int(x))

There were no null values for the gross_us column, but by using value_counts() we could see that there were rows with the value of '--'.  We subsetted out the rows containing the string, and then converted the column to float values.

In [37]:
combined.gross_us.value_counts()
combined = combined[~combined.gross_us.str.contains("--")]
combined.gross_us = combined.gross_us.astype(float)

In [38]:
combined.to_csv('data/combined.csv', index=False)